In [127]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [128]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
data = pd.concat([train, val])

In [129]:
test['dt_of_athrztn']

0              NaN
1       20011228.0
2              NaN
3              NaN
4              NaN
5       20000724.0
6       19920615.0
7           1930.0
8       20010903.0
9       19951104.0
10      20010724.0
11             NaN
12      20021102.0
13      20110801.0
14      19251220.0
15      20120719.0
16      20130308.0
17      19950204.0
18      20051027.0
19      19910625.0
20      20150923.0
21      19970217.0
22             NaN
23      20010329.0
24      20110518.0
25          1921.0
26      20080423.0
27      19341220.0
28      20110105.0
29      20121206.0
           ...    
2927    19940808.0
2928    20011023.0
2929           NaN
2930           NaN
2931    19431220.0
2932    20020322.0
2933    19740428.0
2934    20110127.0
2935    20000306.0
2936    20010730.0
2937    19331220.0
2938    20041206.0
2939    20080317.0
2940    20161028.0
2941    20020704.0
2942           NaN
2943    20091124.0
2944    20000127.0
2945    19980204.0
2946           NaN
2947    19920818.0
2948    1997

In [111]:
train = train.dropna(subset = ['dt_of_athrztn'])
val = val.dropna(subset = ['dt_of_athrztn'])
test = test.dropna(subset = ['dt_of_athrztn'])

In [112]:

train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(str,inplace = True)

train['dt_of_athrztn']=train['dt_of_athrztn'].str[:4]

train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(int,inplace = True)

len(train['dt_of_athrztn'].unique())

171

In [113]:

bins = np.linspace(1900,2019, 20)
train['dt_of_athrztn'] = np.digitize(train['dt_of_athrztn'], bins)


In [114]:
#val['dt_of_athrztn'].fillna(9999,inplace  = True)

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(str,inplace = True)

val['dt_of_athrztn']=val['dt_of_athrztn'].str[:4]

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(int,inplace = True)

len(val['dt_of_athrztn'].unique())

bins = np.linspace(1900,2019, 20)
val['dt_of_athrztn'] = np.digitize(val['dt_of_athrztn'], bins)

#test['dt_of_athrztn'].fillna(9999,inplace  = True)

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(str,inplace = True)

test['dt_of_athrztn']=test['dt_of_athrztn'].str[:4]

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(int,inplace = True)

len(test['dt_of_athrztn'].unique())
bins = np.linspace(1900,2019, 20)
test['dt_of_athrztn'] = np.digitize(test['dt_of_athrztn'], bins)

In [115]:
test = test.drop(['fr_yn'],1)

In [116]:
data['dt_of_fr'] = pd.to_datetime(data['dt_of_fr'])
train['dt_of_fr'] = pd.to_datetime(train['dt_of_fr'])
val['dt_of_fr'] = pd.to_datetime(val['dt_of_fr'])
test['dt_of_fr'] = pd.to_datetime(test['dt_of_fr'])



In [117]:
categorical_cols1 = train.select_dtypes(['object']).columns
for col in categorical_cols1:
    train[col] = pd.Categorical(train[col]).codes


train['DT_hour'] = (train['dt_of_fr'].dt.hour).astype(np.int8)
train['DT_day_week'] = (train['dt_of_fr'].dt.dayofweek).astype(np.int8)
train['DT_day_month'] = (train['dt_of_fr'].dt.day).astype(np.int8)
train['DT_M'] = (train['dt_of_fr'].dt.month).astype(np.int8)
train = train.drop('dt_of_fr', axis = 1)

In [118]:
categorical_cols1 = val.select_dtypes(['object']).columns
for col in categorical_cols1:
    val[col] = pd.Categorical(val[col]).codes


val['DT_hour'] = (val['dt_of_fr'].dt.hour).astype(np.int8)
val['DT_day_week'] = (val['dt_of_fr'].dt.dayofweek).astype(np.int8)
val['DT_day_month'] = (val['dt_of_fr'].dt.day).astype(np.int8)
val['DT_M'] = (val['dt_of_fr'].dt.month).astype(np.int8)
val = val.drop('dt_of_fr', axis = 1)

In [119]:
categorical_cols1 = test.select_dtypes(['object']).columns
for col in categorical_cols1:
    test[col] = pd.Categorical(test[col]).codes


test['DT_hour'] = (test['dt_of_fr'].dt.hour).astype(np.int8)
test['DT_day_week'] = (test['dt_of_fr'].dt.dayofweek).astype(np.int8)
test['DT_day_month'] = (test['dt_of_fr'].dt.day).astype(np.int8)
test['DT_M'] = (test['dt_of_fr'].dt.month).astype(np.int8)
test = test.drop('dt_of_fr', axis = 1)

In [120]:
bins = np.linspace(0,800000, 1000)
train['ttl_ar'] = np.digitize(train['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
val['ttl_ar'] = np.digitize(val['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
test['ttl_ar'] = np.digitize(test['ttl_ar'], bins)


In [21]:
# # Scatter Plot TV
# # x = TV, y = Sales
# train.plot(kind='scatter', x='ttl_grnd_flr', y='fr_yn',alpha = 0.5,color = 'red')
# plt.xlabel('TV')              # label = name of label
# plt.ylabel('Sales')
# plt.title('TV Sales Scatter Plot')            # title = title of plot

In [23]:

# df_all = pd.concat([data,train,val,test])

# categorical_cols = df_all.select_dtypes(['object']).columns
# for col in categorical_cols:
#     df_all[col] = pd.Categorical(df_all[col]).codes

In [24]:
# df_all['DT_hour'] = (df_all['dt_of_fr'].dt.hour).astype(np.int8)
# df_all['DT_day_week'] = (df_all['dt_of_fr'].dt.dayofweek).astype(np.int8)
# df_all['DT_day_month'] = (df_all['dt_of_fr'].dt.day).astype(np.int8)
# df_all['DT_M'] = (df_all['dt_of_fr'].dt.month).astype(np.int8)
# df_all = df_all.drop('dt_of_fr', axis = 1)

In [121]:
X_train = train.drop(['fr_yn'],axis = 1)
X_test = val.drop(['fr_yn'],axis = 1)
y_train = train['fr_yn']
y_test = val['fr_yn']
X = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])

In [122]:

X_train.drop(['id'], axis = 1, inplace = True)
test.drop(['id'], axis = 1, inplace = True)
X_test.drop(['id'], axis = 1, inplace = True)

In [123]:
#모델 생성 및 학습for i in range(300):
    
    import lightgbm as lgb
    Lgb = lgb.LGBMClassifier(n_estimators=18, 
                             silent=False, 
                             random_state =10, 
                             max_depth=18,
                             num_leaves=128,
                             learning_rate = 0.1,
                             objective='binary',
                             metrics ='auc'
                            )
    model = Lgb.fit(X_train, y_train)

#test를 통한 모델 검증
    predictions = model.predict(X_test)
    proba = model.predict_proba(X_test)
    from sklearn.metrics import accuracy_score, roc_auc_score
    from sklearn.metrics import f1_score, recall_score
    score = roc_auc_score(y_test, predictions)
    score2 = accuracy_score(y_test, predictions)
    score3 = f1_score(y_test, predictions)
    score4 = recall_score(y_test, predictions)
#     print('roc_auc_score:',score)
#     print('accuracy_score:',score2)
    print(1,":",'F1', score3)
    #print('recall', score4)

1 : F1 0.5131952902963866


In [39]:
predictions
#18
# roc_auc_score: 0.6914737170414987
# accuracy_score: 0.8153087851551174
#F1 0.5074750830564785

array([0, 0, 0, ..., 0, 0, 1], dtype=int8)

In [40]:
proba

array([[8.63597608e-01, 1.36402392e-01],
       [9.99997053e-01, 2.94707722e-06],
       [8.63618253e-01, 1.36381747e-01],
       ...,
       [9.28619866e-01, 7.13801338e-02],
       [7.92463305e-01, 2.07536695e-01],
       [4.41205862e-01, 5.58794138e-01]])

In [124]:
y = model.predict(test)
y_prob = model.predict_proba(test)


In [125]:
y

array([0, 0, 0, ..., 0, 0, 1], dtype=int8)

In [49]:
y_prob

array([[9.99997535e-01, 2.46466951e-06],
       [2.01446828e-01, 7.98553172e-01],
       [9.99996923e-01, 3.07727009e-06],
       ...,
       [8.85895661e-01, 1.14104339e-01],
       [7.04475316e-01, 2.95524684e-01],
       [5.53848974e-02, 9.44615103e-01]])

In [50]:
y

array([0, 1, 0, ..., 0, 0, 1], dtype=int8)

In [52]:
y_prob2

array([[9.99999938e-01, 6.17229811e-08],
       [7.35424038e-01, 2.64575962e-01],
       [9.99959758e-01, 4.02423327e-05],
       ...,
       [9.96727257e-01, 3.27274313e-03],
       [9.62001801e-01, 3.79981993e-02],
       [6.34868527e-02, 9.36513147e-01]])

In [47]:
y2 = model.predict(test)
y_prob2 = model.predict_proba(test)


In [68]:
a = pd.DataFrame((y_prob+ y_prob)/2)

In [69]:
a['tf'] = a[0] < a[1]

In [73]:
a['tf']

0       False
1        True
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15       True
16       True
17       True
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2927     True
2928    False
2929    False
2930    False
2931    False
2932     True
2933    False
2934    False
2935    False
2936    False
2937    False
2938    False
2939    False
2940    False
2941     True
2942    False
2943    False
2944    False
2945    False
2946    False
2947    False
2948     True
2949     True
2950    False
2951     True
2952    False
2953    False
2954    False
2955    False
2956     True
Name: tf, Length: 2957, dtype: bool

In [63]:
a['tf'] = a['tf'].map({False:'N',True:'Y'})

In [76]:
sub['fr_yn'] = a['tf']

In [126]:
sub['fr_yn'] = y
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.head()

ValueError: Length of values does not match length of index

In [32]:
sub.to_csv('tmddnjs94_화재예측과제_진우20파라미터.csv', index=False)

In [74]:
a['tf']
a['tf'] = pd.Categorical(a['tf']).codes

In [78]:
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})

In [85]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957 entries, 0 to 2956
Data columns (total 1 columns):
fr_yn    2957 non-null object
dtypes: object(1)
memory usage: 23.2+ KB
